In [13]:
import sys  
sys.path.insert(0, './')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import seaborn as sns
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [14]:
# Filtered to just Bananas
df = pd.read_csv('../round3/data/prices_round_3_day_1.csv', index_col='timestamp', delimiter=';')
df = df.loc[df['product']=='BERRIES']
df = df[['mid_price']].rename(columns={'mid_price': 'Value'})
df.head()

,Value
timestamp,
0,3919.5
100,3919.5
200,3919.5
300,3919.5
400,3920.5


In [21]:
class SARIMAX:
    def __init__(self, order=(0, 0, 0), seasonal_order=(0, 0, 0, 0), trend=None):
        self.p, self.d, self.q = order
        self.P, self.D, self.Q, self.s = seasonal_order
        self.trend = trend
        self.params = None

    def fit(self, X):
        nobs = X.shape[0]
        y = X[self.s:]
        X = self._build_matrix(X)
        self.params = np.linalg.lstsq(X, y, rcond=None)[0]

    def predict(self, n_periods):
        y_pred = np.zeros(n_periods)
        x_pred = np.zeros(self.p + self.s * self.P)
        for i in range(n_periods):
            x_pred[0] = 1  # set the intercept term
            for j in range(self.p):
                x_pred[j + 1] = y_pred[i - self.d + j] if i >= self.d - 1 else 0
            for j in range(self.P):
                x_pred[self.p + j * self.s + 1:self.p + (j + 1) * self.s + 1] = (
                    y_pred[i - self.D * self.s + j * self.s : i - self.D * self.s + (j + 1) * self.s]
                    if i >= self.D * self.s - 1
                    else 0
                )
            y_pred[i] = np.dot(self.params, x_pred)
        return y_pred

    def _build_matrix(self, X):
        # First, compute the differences and seasonal differences of X
        X_diff = np.diff(X, n=self.d)
        X_diff = np.concatenate(([X[self.d - 1]], X_diff))

        if self.D > 0:
            X_seasonal_diff = np.diff(X, n=self.s * self.D, axis=0)
            X_seasonal_diff = np.concatenate(
                [X[self.s * self.D :], X_seasonal_diff], axis=0
            )
        else:
            X_seasonal_diff = X

        # Construct the design matrix
        nobs = X.shape[0]
        X_mat = np.zeros((nobs - self.p, self.p + self.P * self.s))

        for i in range(self.p, nobs):
            # Autoregressive terms
            X_mat[i - self.p, : self.p] = X_seasonal_diff[i - 1 : i - self.p - 1 : -1]

            # Moving average terms
            if self.q > 0:
                X_mat[i - self.p, self.p : self.p + self.q] = (
                    X_diff[i - 1 : i - self.q - 1 : -1]
                )

            # Seasonal autoregressive terms
            for j in range(self.P):
                X_mat[i - self.p, self.p + j * self.s : self.p + (j + 1) * self.s] = (
                    X_seasonal_diff[i - self.s * (j + 1) : i - self.s * j - 1 : -1]
                )

            # Seasonal moving average terms
            if self.Q > 0:
                X_mat[i - self.p, self.p + self.P * self.s :] = (
                    X_seasonal_diff[i - self.s : i - self.s * self.Q - 1 : -self.s][::-1]
                )

        return X_mat


In [22]:
# Generate a sample time series
np.random.seed(123)
time_series = np.random.normal(loc=0, scale=1, size=1000)

# Create a SARIMAX model with (1, 1, 1) x (1, 0, 1, 12) orders and a constant trend
model = SARIMAX(order=(1, 1, 1), seasonal_order=(1, 0, 1, 12), trend="c")

# Fit the model to the time series
model.fit(time_series)

# Generate forecasts for the next 12 time periods
n_periods = 12
forecast = model.predict(n_periods=n_periods)[-n_periods:]

# Plot the time series and the forecasts
plt.plot(time_series, label="Observed")
plt.plot(np.arange(len(time_series) - n_periods, len(time_series)), forecast, label="Forecast")
plt.legend()
plt.show()

ValueError: could not broadcast input array from shape (0,) into shape (1,)

In [28]:
class ARIMA:
    def __init__(self, p, d, q):
        self.p = p # AR order
        self.d = d # differencing order
        self.q = q # MA order
        
        self.coeffs = None # coefficients of the model
        self.residuals = None # residuals of the model
        
    def fit(self, data):
        # Perform differencing
        diff_data = np.diff(data, n=self.d)
        
        # Initialize variables
        n = len(diff_data)
        y = diff_data[self.p:]
        X = np.zeros((n - self.p, self.p + self.q))
        
        # Populate X matrix with lags of y and errors
        for i in range(self.p):
            X[:, i] = diff_data[self.p - 1 - i:n - 1 - i]
        for i in range(self.q):
            X[:, self.p + i] = self.residuals[self.p - 1 - i:n - 1 - i]
        
        # Compute coefficients using least squares regression
        self.coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
        
        # Compute residuals
        self.residuals = np.zeros(n - self.p)
        self.residuals[0] = diff_data[self.p] - X[0, :].dot(self.coeffs)
        for i in range(1, n - self.p):
            self.residuals[i] = diff_data[self.p + i] - X[i, :].dot(self.coeffs)
        
    def predict(self, n):
        # Predict n future values
        pred = np.zeros(n)
        for i in range(n):
            # Compute next value using the model coefficients and previous residuals
            prev_vals = np.concatenate((self.residuals[-self.p:], pred[:i]))
            pred[i] = self.coeffs[:self.p].dot(prev_vals) + self.coeffs[self.p:].dot(self.residuals[-self.q:])
        
        # Perform inverse differencing
        for i in range(1, self.d + 1):
            pred = np.concatenate(([pred[0]], np.cumsum(pred)[i:] - np.cumsum(pred)[:-i]))
            
        return pred


In [ ]:
# Generate a sample time series
np.random.seed(123)
time_series = np.random.normal(loc=0, scale=1, size=1000)

# Create a SARIMAX model with (1, 1, 1) x (1, 0, 1, 12) orders and a constant trend
model = ARIMA(order=(1, 0, 1)

# Fit the model to the time series
model.fit(time_series)

# Generate forecasts for the next 12 time periods
n_periods = 12
forecast = model.predict(n_periods=n_periods)[-n_periods:]

# Plot the time series and the forecasts
plt.plot(time_series, label="Observed")
plt.plot(np.arange(len(time_series) - n_periods, len(time_series)), forecast, label="Forecast")
plt.legend()
plt.show()